In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch.nn.functional as F

In [2]:
DATA_PATH = "../data/astro_dataset_maxia/astro_dataset_maxia"

In [3]:
image_sz = 128 
batch_sz = 32

In [4]:
from torchvision import transforms

# Preprocessing: Image Resize to be same size, for each image
# Preprocessing: Randomly Rotating Images about y-axis
# Preprocessing: Normalizing pixel values

train_transform = transforms.Compose([
    transforms.Resize((image_sz, image_sz)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

eval_transform = transforms.Compose([
    transforms.Resize((image_sz, image_sz)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

In [5]:
train_data = datasets.ImageFolder(os.path.join(DATA_PATH, "training"), transform=train_transform)
test_data  = datasets.ImageFolder(os.path.join(DATA_PATH, "test"), transform=eval_transform)
val_data  = datasets.ImageFolder(os.path.join(DATA_PATH, "validation"), transform=eval_transform)

print("Classes:", train_data.classes)
print("Number of Classes:", len(train_data.classes))
print("Training Samples:", len(train_data))
print("Val Samples:", len(val_data))
print("Test Samples:", len(test_data))
print("Total Samples:", len(train_data)+len(val_data)+len(test_data))

Classes: ['asteroid', 'black_hole', 'earth', 'galaxy', 'jupiter', 'mars', 'mercury', 'neptune', 'pluto', 'saturn', 'uranus', 'venus']
Number of Classes: 12
Training Samples: 2416
Val Samples: 658
Test Samples: 345
Total Samples: 3419


In [6]:
class CNN(nn.Module):
    def __init__(self, num_classes, img_sz):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, padding=1, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, padding=1, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 256, padding=1, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)

        total_nodes = 256*int(img_sz/8)**2

        self.layer1 = nn.Linear(total_nodes, 256)
        self.dropout = nn.Dropout(0.35)

        self.layer2 = nn.Linear(256, num_classes)

    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)

        return x


In [7]:
class CNN_Ultra(nn.Module):
    def __init__(self, num_classes, img_sz):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, padding=1, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, padding=1, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 256, padding=1, kernel_size=3)
        self.conv4 = nn.Conv2d(256, 512,  padding=1, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)

        total_nodes = 512*int(img_sz/8)**2

        self.layer1 = nn.Linear(total_nodes, 512)
        self.dropout = nn.Dropout(0.3)

        self.layer2 = nn.Linear(512, num_classes)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = F.relu(x)

        x = x.view(x.size(0), -1)

        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)

        return x


In [8]:
class CNN_Turbo(nn.Module):
    def __init__(self, num_classes, img_sz):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, padding=1, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)

        total_nodes = 64*int(img_sz/2)**2

        self.layer1 = nn.Linear(total_nodes, 64)
        self.dropout = nn.Dropout(0.15)

        self.layer2 = nn.Linear(64, num_classes)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        x = self.layer1(x)
        x = F.relu(x)
        
        x = self.dropout(x)

        x = self.layer2(x)

        return x


In [9]:
class CNN_Lite(nn.Module):
    def __init__(self, num_classes, img_sz):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, padding=1, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, padding=1, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)

        total_nodes = 128*int(img_sz/4)**2

        self.layer1 = nn.Linear(total_nodes, 128)
        self.dropout = nn.Dropout(0.2)

        self.layer2 = nn.Linear(128, num_classes)

    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)

        return x


In [10]:
train_loader = DataLoader(train_data, batch_size=batch_sz, shuffle=True)
val_loader   = DataLoader(val_data,   batch_size=batch_sz, shuffle=False)
test_loader  = DataLoader(test_data,  batch_size=batch_sz, shuffle=False)

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = CNN(len(train_data.classes), image_sz).to(device)

In [12]:
import torch.optim as optim

def cnn_trainer(CNNmodel, epochs, lamb=0):
    model = CNNmodel(num_classes=len(train_data.classes), img_sz=image_sz).to(device)
    print(device)
    
    histmap = {"train_loss": [],
               "train_accuracy": [],  
               "val_loss": [], 
               "val_accuracy": []
    }

    adam = optim.Adam(model.parameters(), lr=1e-3, weight_decay=lamb)
    objective = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        correct = 0
        total = 0
        cur_loss = 0.0


        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            adam.zero_grad()
            outputs = model(images)
            loss = objective(outputs, labels)
            loss.backward()
            adam.step()

            cur_loss += loss.item() * labels.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = cur_loss/total
        train_accuracy = correct/total

        model.eval()
        val_correct = 0
        val_total = 0
        val_loss_sum = 0


        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = objective(outputs, labels)
                val_loss_sum += loss.item() * labels.size(0)

                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        val_loss = val_loss_sum / val_total
        val_accuracy = val_correct / val_total

        histmap["train_loss"].append(train_loss)
        histmap["train_accuracy"].append(train_accuracy)

        histmap["val_loss"].append(val_loss)
        histmap["val_accuracy"].append(val_accuracy)

        print(f"lambda: {lamb} and epoch: {epoch} of {epochs}, train accuracy: {train_accuracy} and val accuracy: {val_accuracy}")

    return model, histmap


In [ ]:
lamb_exp = 0.01
model_test, hist_test = cnn_trainer(CNN, 3, lamb_exp)

cuda
lambda: 0.01 and epoch: 1 of 3, train accuracy: 0.4913079470198676 and val accuracy: 0.621580547112462
lambda: 0.01 and epoch: 2 of 3, train accuracy: 0.6870860927152318 and val accuracy: 0.7203647416413373
lambda: 0.01 and epoch: 3 of 3, train accuracy: 0.7189569536423841 and val accuracy: 0.7583586626139818


In [13]:
lambs = [0, 0.01, 0.001, 0.005, 0.0001, 0.0005]

In [55]:
res = {}

for lamb in lambs:
    filename = "CNN_lambda-"+str(lamb)+".pth"
    model, histmap = cnn_trainer(CNN, 14, lamb)
    res[lamb] = histmap

    torch.save(model.state_dict(), filename)
    print("Saved:", filename)


cuda
lambda: 0 and epoch: 1 of 14, train accuracy: 0.4859271523178808 and val accuracy: 0.7629179331306991
lambda: 0 and epoch: 2 of 14, train accuracy: 0.7379966887417219 and val accuracy: 0.8343465045592705
lambda: 0 and epoch: 3 of 14, train accuracy: 0.8133278145695364 and val accuracy: 0.851063829787234
lambda: 0 and epoch: 4 of 14, train accuracy: 0.847682119205298 and val accuracy: 0.8617021276595744
lambda: 0 and epoch: 5 of 14, train accuracy: 0.8733443708609272 and val accuracy: 0.8829787234042553
lambda: 0 and epoch: 6 of 14, train accuracy: 0.8973509933774835 and val accuracy: 0.8844984802431611
lambda: 0 and epoch: 7 of 14, train accuracy: 0.9126655629139073 and val accuracy: 0.8936170212765957
lambda: 0 and epoch: 8 of 14, train accuracy: 0.9246688741721855 and val accuracy: 0.9027355623100304
lambda: 0 and epoch: 9 of 14, train accuracy: 0.9387417218543046 and val accuracy: 0.9027355623100304
lambda: 0 and epoch: 10 of 14, train accuracy: 0.9391556291390728 and val accur

In [ ]:
res

{0: {'train_loss': [1.5898193322270122,
   0.792324359843273,
   0.574714989259543,
   0.4596526308959683,
   0.37858685130709846,
   0.31432364737948043,
   0.27391967680675305,
   0.23979092961707651,
   0.1787006850570243,
   0.18992350428112295,
   0.14435225552478373,
   0.12347792393053013,
   0.1229284115826452,
   0.09890862847894233],
  'train_accuracy': [0.4859271523178808,
   0.7379966887417219,
   0.8133278145695364,
   0.847682119205298,
   0.8733443708609272,
   0.8973509933774835,
   0.9126655629139073,
   0.9246688741721855,
   0.9387417218543046,
   0.9391556291390728,
   0.9449503311258278,
   0.9577814569536424,
   0.9598509933774835,
   0.9664735099337748],
  'val_loss': [0.8394283245580899,
   0.5796191444150582,
   0.47783586797856453,
   0.4487835445574352,
   0.41606758768163554,
   0.4070993214321965,
   0.30976974455169815,
   0.31975000576439205,
   0.39423588729013426,
   0.34671219677894694,
   0.3770630520397456,
   0.2777733909541449,
   0.299394739618620

In [72]:
for lamb, ls in res.items():
    best_accuracy = ls['val_accuracy'][-1]
    print(lamb, best_accuracy)

0 0.9194528875379939
0.01 0.8768996960486323
0.001 0.9346504559270516
0.005 0.8860182370820668
0.0001 0.9331306990881459
0.0005 0.9194528875379939


In [69]:
# CNN-Turbo

res_turbo = {}

for lamb in lambs:
    filename = "CNN-Turbo_lambda-"+str(lamb)+".pth"
    model, histmap = cnn_trainer(CNN_Turbo, 14, lamb)
    res_turbo[lamb] = histmap

    torch.save(model.state_dict(), filename)
    print("Saved:", filename)


cuda
lambda: 0 and epoch: 1 of 14, train accuracy: 0.21812913907284767 and val accuracy: 0.29027355623100304
lambda: 0 and epoch: 2 of 14, train accuracy: 0.29759933774834435 and val accuracy: 0.4635258358662614
lambda: 0 and epoch: 3 of 14, train accuracy: 0.37251655629139074 and val accuracy: 0.46656534954407297
lambda: 0 and epoch: 4 of 14, train accuracy: 0.3961092715231788 and val accuracy: 0.45288753799392095
lambda: 0 and epoch: 5 of 14, train accuracy: 0.4019039735099338 and val accuracy: 0.49848024316109424
lambda: 0 and epoch: 6 of 14, train accuracy: 0.43543046357615894 and val accuracy: 0.5516717325227963
lambda: 0 and epoch: 7 of 14, train accuracy: 0.4586092715231788 and val accuracy: 0.5577507598784195
lambda: 0 and epoch: 8 of 14, train accuracy: 0.5235927152317881 and val accuracy: 0.6975683890577508
lambda: 0 and epoch: 9 of 14, train accuracy: 0.5778145695364238 and val accuracy: 0.723404255319149
lambda: 0 and epoch: 10 of 14, train accuracy: 0.5844370860927153 and 

In [71]:
for lamb, ls in res_turbo.items():
    best_accuracy = ls['val_accuracy'][-1]
    print(lamb, best_accuracy)

0 0.7462006079027356
0.01 0.7933130699088146
0.001 0.7188449848024316
0.005 0.7355623100303952
0.0001 0.6717325227963525
0.0005 0.8191489361702128


In [73]:
res_ultra = {}

for lamb in lambs:
    filename = "CNN-Ultra_lambda-"+str(lamb)+".pth"
    model, histmap = cnn_trainer(CNN_Ultra, 14, lamb)
    res_ultra[lamb] = histmap

    torch.save(model.state_dict(), filename)
    print("Saved:", filename)


cuda
lambda: 0 and epoch: 1 of 14, train accuracy: 0.4586092715231788 and val accuracy: 0.6930091185410334
lambda: 0 and epoch: 2 of 14, train accuracy: 0.7392384105960265 and val accuracy: 0.817629179331307
lambda: 0 and epoch: 3 of 14, train accuracy: 0.8182947019867549 and val accuracy: 0.8358662613981763
lambda: 0 and epoch: 4 of 14, train accuracy: 0.8551324503311258 and val accuracy: 0.8677811550151976
lambda: 0 and epoch: 5 of 14, train accuracy: 0.8754139072847682 and val accuracy: 0.8753799392097265
lambda: 0 and epoch: 6 of 14, train accuracy: 0.8965231788079471 and val accuracy: 0.89209726443769
lambda: 0 and epoch: 7 of 14, train accuracy: 0.9048013245033113 and val accuracy: 0.9027355623100304
lambda: 0 and epoch: 8 of 14, train accuracy: 0.9325331125827815 and val accuracy: 0.8996960486322189
lambda: 0 and epoch: 9 of 14, train accuracy: 0.9466059602649006 and val accuracy: 0.9376899696048632
lambda: 0 and epoch: 10 of 14, train accuracy: 0.9461920529801324 and val accura

In [74]:
for lamb, ls in res_ultra.items():
    best_accuracy = ls['val_accuracy'][-1]
    print(lamb, best_accuracy)

0 0.9346504559270516
0.01 0.851063829787234
0.001 0.8936170212765957
0.005 0.8844984802431611
0.0001 0.9133738601823708
0.0005 0.8996960486322189


In [ ]:
res_lite = {}

for lamb in lambs:
    filename = "CNN-Lite_lambda-"+str(lamb)+".pth"
    model, histmap = cnn_trainer(CNN_Lite, 14, lamb)
    res_lite[lamb] = histmap

    torch.save(model.state_dict(), filename)
    print("Saved:", filename)


cuda
lambda: 0 and epoch: 1 of 14, train accuracy: 0.5107615894039735 and val accuracy: 0.7477203647416414
lambda: 0 and epoch: 2 of 14, train accuracy: 0.7632450331125827 and val accuracy: 0.8191489361702128
lambda: 0 and epoch: 3 of 14, train accuracy: 0.828228476821192 and val accuracy: 0.8541033434650456
lambda: 0 and epoch: 4 of 14, train accuracy: 0.8551324503311258 and val accuracy: 0.8677811550151976
lambda: 0 and epoch: 5 of 14, train accuracy: 0.8882450331125827 and val accuracy: 0.8662613981762918
lambda: 0 and epoch: 6 of 14, train accuracy: 0.8932119205298014 and val accuracy: 0.8875379939209727
lambda: 0 and epoch: 7 of 14, train accuracy: 0.9242549668874173 and val accuracy: 0.8844984802431611
lambda: 0 and epoch: 8 of 14, train accuracy: 0.9337748344370861 and val accuracy: 0.8996960486322189
lambda: 0 and epoch: 9 of 14, train accuracy: 0.9432947019867549 and val accuracy: 0.8905775075987842
lambda: 0 and epoch: 10 of 14, train accuracy: 0.9540562913907285 and val accu

In [ ]:
for lamb, ls in res_lite.items():
    best_accuracy = ls['val_accuracy'][-1]
    print(lamb, best_accuracy)

0 0.9118541033434651
0.01 0.8875379939209727
0.001 0.8951367781155015
0.005 0.8844984802431611
0.0001 0.9300911854103343
0.0005 0.9103343465045592
